*Stanislav Borysov [stabo@dtu.dk], DTU Management*
# Advanced Business Analytics

## Web Data Mining - Part 2: Weather data

A lot of valuable data can be collected from online sources. It can be used to get insights, make predictions, etc. In this notebook, we're going to practice data collection from online sources using API. Particularly, we will collect social media data from Twitter and weather data from Weatherbit.

### 1. Getting weather data from https://www.weatherbit.io/

#### 1.1. Creating developer account

1. Register on the website
2. Get API key in the dashboard https://www.weatherbit.io/account/dashboard
**If you see the following message: "Your API Key is still provisioning. Provisioning can take up to 30 mins.", go to Part 3 until your key is ready.**

Note that the free plan includes:

- 500 calls/day
- 500 historical calls/day (trial)
- 1 month historical
- 16 day forecasts
- 48 hour forecasts (trial)
- Air Quality / Energy API (trial)
- Non-Commercial use only
- 95.0% Uptime
- Data update delay: 1 hour

So use your API calls carefully!

API documentation: https://www.weatherbit.io/api

#### 1.2. Getting current weather
https://www.weatherbit.io/api/weather-current

Current weather can be obtained as a response from such kind of url = 'https://api.weatherbit.io/v2.0/current?lat=X&lon=Y&key=api_key'

Let's get the weather for DTU. For that, we need the coordinates of DTU. You can use many different sources to get the coordinates, for example, Wikipedia or Google maps. For instance, coordinates of the little mermaid are ```lat=55.693019``` and ```lon=12.599266```.

In [ ]:
api_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
dtu_lat = '' 
dtu_lon = ''

Let's generate the request url and try it in the browser first. To format strings, the ```format``` function is very handy:
```
'lat={}&lon={}&key={}'.format(dtu_lat, dtu_lon, api_key)
```
will insert these three values into the string

In [ ]:
# ...

Copy it to the browser and check if you get any response. To get response in python, we can use the ```urllib``` library: 
```
import urllib.request
contents = urllib.request.urlopen("http://example.com/foo/bar").read().decode("utf-8")
```

In [ ]:
# ...

As you can see, the response is a string, so let's parse it into a dictionary using the ```json``` library

In [ ]:
# ...

Look outside the window. Is it what you expected? :)

#### 1.3. Getting historical data

**UNFORTUNATELY, WEATHERBIT.IO STOPPED PROVIDING ACCESS TO HISTORICAL DATA WITHOUT SIGNING UP FOR A TRIAL VERSION. YOU CAN SKIP THIS SUBSECTION AND GO TO 1.4. YOU CAN ALSO CHECK THE NOTEBOOK WITH SOLUTIONS (Part 2 - Weather-solutions.ipynb).**

Historical weather data is usually more difficult to get for free because it has business value. Weatherbit.io offers 200 calls per day for the free account. It's not much but still enough for our purposes. It's better to use calls for 10 days not to exceed the limits at once (don't worry, the data for the further analysis will be provided).

To get historical weather, use the response from the following URL:

```https://api.weatherbit.io/v2.0/history/daily?city=Raleigh,NC&start_date=YYYY-MM-DD&end_date=YYYY-MM-DD&key=API_KEY```

You might want to check the full documentation here: https://www.weatherbit.io/api/weather-history-daily

The url requires specifying ```start_date``` and ```end_date```. Note that under the free plan the distance between them should not exceed one day, e.g. ```start_date=2018-09-17&end_date=2018-09-18```. So, let's generate a list of the dates for the requests first. In this case, our old good friend, the ```datatime``` library, will help us again. Use the ```timedelta``` function to generate the previous day:
```
import datetime
date_1 = "2018-08-31"
date_0 = datetime.datetime.strptime(date_1, "%Y-%m-%d") - datetime.timedelta(days=1)
date_0 = date_0.strftime("%Y-%m-%d") 
print(date_0) # will print "2018-08-30"
```

In [ ]:
# ...

Now, it's time to collect our data. Get a list of 10 historical weather observations and print it. Remember, you need to make a separate call to the website for each pair of dates. Use the json.loads(response) to parse the response string as in the previous exercise.

In [ ]:
# ...

Print the number of retrieved days and the first observation to check if the results are correct.

In [ ]:
# ...

Great! Now, you can access the data using the python dictionaries. For example, the wind speed for the first observation can be accessed as 

In [ ]:
print(observations[0]['data'][0]['max_wind_spd'])

It is convenient to work with this type of data as a pandas dataframe. There are many ways how to create dataframe from the data. We'll use dictionaries, so let's do some preparation first. Fill the following dictionary with the values from the retrieved data:

In [ ]:
data_dict = {
    'datetime': [],
    'state_code': [],
    'country_code': [],
    'lat': [],
    'lon': [],
    'city_name': [],
    'station_id': [],
    'ts': [],
    'rh': [],
    'clouds': [],
    'pres': [],
    'snow': [],
    'snow_depth': [],
    'precip': [],
    'wind_spd': [],
    'wind_dir': [],
    'max_wind_spd': [],
    'max_wind_dir': [],
    'wind_gust_spd': [],
    'temp': [],
    'max_temp': [],
    'min_temp': [],
}

Now, ```import pandas as pd``` and use the ```df = pd.DataFrame.from_dict(data_dict)``` to create the dataframe. It's also convenient to set index to the ```'datetime'``` column. Finaly, use ```df.info()``` and ```df.head()``` to check if the created dataframe is correct.

In [ ]:
# ...

Cool! Save the dataframe into a csv file using ```df.to_csv('my_dtu_weather.csv')```

In [ ]:
# ...

#### 1.4. Plotting the data

Read the data to dataframe from the "dtu_weather_199.csv" file. You can use the ```pd.read_csv``` function with the following parameters: ```index_col='datetime'``` and ```parse_dates=True```.

In [ ]:
# ...

Let's make some plots. Plot maximum, minimum and average temperature.

In [ ]:
# ...

To plot wind speed depending on wind direction, polar plots are helpful. Check https://matplotlib.org/gallery/pie_and_polar_charts/polar_demo.html for the details.

Hints:
- polar plots are using radians instead of degrees, so create another column ```'wind_dir_rad'``` using a function from numpy ```np.radians()```
- to align 0 to the north and enforce clock-wise direction, you can use  ```ax.set_theta_zero_location('N')``` and 
```ax.set_theta_direction(-1)``` respectively

In [ ]:
# ...

The wind data is a bit scattered. A better representation might be to plot mean and standard deviation for each wind direction. For example, the mean can be computed as 
```
mean = df['wind_spd'].groupby([df['wind_dir_rad']]).mean()
```

Hint: To plot standard deviation, you can plot the two lines corresponding to mean +/- std

In [ ]:
# ...

It seems that we have too few data points for each wind direction to get useful insights. Let's try to aggregate data by the 16 main wind directions (North, North-NorthEast, NorthEast, etc). We can use the ```round_degrees()``` function defined below to round the wind direction. Create new column 'wind_dir_agg_rad' in the dataframe using this function.

Sidequest: Can you rewrite this 'ugly' function to make it more 'elegant'?

In [ ]:
def round_degrees(val):
    if val >= 348.75 or val < 11.25:
         return 0.0
    elif val >= 11.25 and val < 33.75:
         return 22.5
    elif val >= 33.75 and val < 56.25:
         return 45.0
    elif val >= 56.25 and val < 78.75:
         return 67.5
    elif val >= 78.75 and val < 101.25:
         return 90.0
    elif val >= 101.25 and val < 123.75:
         return 112.5
    elif val >= 123.75 and val < 146.25:
         return 135.0
    elif val >= 146.25 and val < 168.75:
         return 157.5
    elif val >= 168.75 and val < 191.25:
         return 180.0
    elif val >= 191.25 and val < 213.75:
         return 202.5
    elif val >= 213.75 and val < 236.25:
         return 225.0
    elif val >= 236.25 and val < 258.75:
         return 247.5
    elif val >= 258.75 and val < 281.25:
         return 270.0
    elif val >= 281.25 and val < 303.75:
         return 292.5
    elif val >= 303.75 and val < 326.25:
         return 315.0
    elif val >= 326.25 and val < 348.75:
         return 337.5

In [ ]:
# ...

Finally, let's try again to plot the aggregated values.

In [ ]:
# ...

Much better! We can see that the most strong wind direction was East-NorthEast (ENE or 67.5 degrees). Also, there was no wind from the north. Might it be related to the great summer we had that year in Denmark?